In [9]:
# 🔹 1. 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install ultralytics

In [11]:
from ultralytics import YOLO
import os
import cv2
import numpy as np


In [12]:
base_dir = "/content/drive/MyDrive/AIProjectYOLO"

In [17]:
import os

model_path = "/content/drive/MyDrive/AIProjectYOLO/runs/yolov11n_plate4/weights/best.pt"

print("模型是否存在？", os.path.exists(model_path))

模型是否存在？ True


In [24]:
# ---------- 參數區 ----------
model_path = '/content/drive/MyDrive/AIProjectYOLO/runs/yolov11n_plate4/weights/best.pt'  # 你的best.pt路徑
image_folder = '/content/drive/MyDrive/AIProjectYOLO/images/val'             # 驗證圖片資料夾
label_folder = '/content/drive/MyDrive/AIProjectYOLO/labels/val'             # YOLO格式標註資料夾
iou_threshold = 0.5                               # 若要判定TP可使用
# ----------------------------


In [26]:
def yolo_to_xyxy(box, img_w, img_h):
    # 預期 box 是長度為 5 的列表: class, cx, cy, w, h
    try:
        cls, cx, cy, w, h = map(float, box[:5])  # 只取前5個欄位
    except:
        return None  # 傳回 None 代表轉換失敗，主程式會自動略過

    x1 = (cx - w / 2) * img_w
    y1 = (cy - h / 2) * img_h
    x2 = (cx + w / 2) * img_w
    y2 = (cy + h / 2) * img_h
    return [x1, y1, x2, y2]

def compute_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    if interArea == 0:
        return 0.0
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return interArea / (box1Area + box2Area - interArea)

# 載入模型
model = YOLO(model_path)

ious = []

# 遍歷圖片
for file in os.listdir(image_folder):
    if not (file.endswith(".jpg") or file.endswith(".png")):
        continue

    img_path = os.path.join(image_folder, file)
    label_path = os.path.join(label_folder, file.replace(".jpg", ".txt").replace(".png", ".txt"))

    if not os.path.exists(label_path):
        continue

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Ground Truth 轉換（安全處理，跳過格式錯誤行）
    gt_boxes = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # 欄位不夠跳過
            box = yolo_to_xyxy(parts, w, h)
            if box is not None:
                gt_boxes.append(box)

    # 模型預測
    results = model(img_path)[0]
    preds = results.boxes.xyxy.cpu().numpy()

    # 每個GT與所有預測比IOU，取最大
    for gt in gt_boxes:
        ious_this_gt = [compute_iou(gt, pred) for pred in preds]
        if ious_this_gt:
            ious.append(max(ious_this_gt))



image 1/1 /content/drive/MyDrive/AIProjectYOLO/images/val/000682_jpg.rf.2b97950e63cd0d800b74278f27cc2e6b.jpg: 640x640 1 plate, 268.7ms
Speed: 6.3ms preprocess, 268.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/AIProjectYOLO/images/val/001702_jpg.rf.24e780a7834d0115c3c2264b038aa7da.jpg: 640x640 1 plate, 329.2ms
Speed: 5.5ms preprocess, 329.2ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/AIProjectYOLO/images/val/000475_jpg.rf.2a2b028f57cd644268d2a4514a728513.jpg: 640x640 1 plate, 398.3ms
Speed: 13.3ms preprocess, 398.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/AIProjectYOLO/images/val/001678_jpg.rf.74c9b3fbb7c153a7f19059230e90b6a7.jpg: 640x640 1 plate, 393.8ms
Speed: 7.7ms preprocess, 393.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/AIProjectYOLO/images/val/0006

In [27]:
# 輸出結果
if ious:
    print(f"\n📊 所有 Ground Truth 平均 IOU = {np.mean(ious):.4f}（共 {len(ious)} 組配對）")
else:
    print("⚠️ 沒有匹配到任何 Ground Truth 和預測框。請檢查標註格式或圖片路徑。")


📊 所有 Ground Truth 平均 IOU = 0.9236（共 492 組配對）
